## Parse image through an LLM (GPT4/Claude/Gemini)
From our initial test GPT4-turbo seems to work fine.

In [ ]:
from scripts.LLMToSchematics import image_to_schematics

LLM_output= image_to_schematics("testImages/led_array.jpeg")
print(LLM_output)

# GPT Output processing

#### Save and load the json from LLM output, so that you can save API requests during development :)

In [ ]:

# save the result dictionary to a file
import json
with open('result.json', 'w') as f:
    json.dump(LLM_output, f, indent=4)



In [ ]:
# load the dictionary from the file
import json
with open('result.json', 'r') as f:
    result = json.load(f)

#### Clean data

In [ ]:
# iterate through the list of dicts results and remove empty dicts
for i in range(len(result['detected_components'])):
    if not result['detected_components'][i]:
        del result['detected_components'][i]
for component in result["detected_components"]:
    print(component["lib_id"])

#### Scale component coordinates for KiCAD schematics

In [ ]:
from scripts import kicad_utils

kicad_schematic_path = "testProject/testProject.kicad_sch"
list_of_component_dict =[]


for symbol in result["detected_components"]:
    print(symbol["lib_id"], symbol["x"], symbol["y"], symbol["angle"], symbol["reference"])
    
    # Add the component to the list
    list_of_component_dict.append({"lib_id": kicad_utils.match_libId(symbol["lib_id"]), "x": symbol["x"], "y": symbol["y"], "angle": symbol["angle"], "reference_name": symbol["reference"]})

print("List of component dictionaries:")
print(list_of_component_dict)
# Accepts a list of component dictionaries and a scaling factor
def scale_components(components, scaling_factor):
    # Find the minimum and maximum x and y coordinates
    min_x = min(component['x'] for component in components)
    max_x = max(component['x'] for component in components)
    min_y = min(component['y'] for component in components)
    max_y = max(component['y'] for component in components)
    
    # Scale the components
    scaled_components = []
    for component in components:
        scaled_x = min_x + (component['x'] - min_x) * scaling_factor
        scaled_y = min_y + (component['y'] - min_y) * scaling_factor
        scaled_component = component.copy()
        scaled_component['x'] = int(scaled_x)
        scaled_component['y'] = int(scaled_y)
        scaled_components.append(scaled_component)
    
    return scaled_components


scaled_components = scale_components(list_of_component_dict, 0.2)
print("Scaled Components:")
print(scaled_components)




#### Hardcoded temporary fixes
- Resistors are always rotated by 90 degrees, since the symbol library describes them in a vertical orientation

In [ ]:


# Change angle t0 90 for all resistors having angle 0 and 0 for all resistors having angle 90
for component in scaled_components:
    if component["lib_id"] == "Device:R":
        if component["angle"] == 0:
            component["angle"] = 90
        else:
            component["angle"] = 0

print("Components after changing angles:")
print(scaled_components)

# Add symbols to schematic!

In [ ]:
from scripts import kicad_utils
kicad_utils.modify_kicad_sch_file(components = scaled_components, file_path=kicad_schematic_path)

In [ ]:
import skip

def find_component_in_schem(component_reference, skip_schematic):
    for symbol in skip_schematic.symbol:
        if symbol.property.Reference.value == component_reference:
            return symbol

schem = skip.Schematic("testProject/testProject.kicad_sch")
wire_list = []

# make a copy of detected connections
all_connections = result["component_connections"].copy()

# Iterate through the list of generated components in kicad schematic
for curr_component in schem.symbol:
    print(curr_component.property.Reference.value)
    curr_component_ref = curr_component.property.Reference.value
    # Iterate through the list of detected connections
    for curr_connection in all_connections:
        # If curr component is compA in curr connection
        if curr_connection['A_ref'] == curr_component_ref:
            try:
                curr_component_A_pin = curr_component.pin[curr_connection['A_pin'] - 1]
                # Find component B
                curr_component_B = find_component_in_schem(curr_connection['B_ref'], schem)
                curr_component_B_pin = curr_component_B.pin[curr_connection['B_pin'] - 1]
                # Add a wire to connect
            
                wire_list.append({"x": curr_component_A_pin.location.x, "y": curr_component_A_pin.location.y,
                                  "end_x": curr_component_B_pin.location.x, "end_y": curr_component_B_pin.location.y})
            except:
                print("Skipped a wire")
                continue



#### Split diagonal wires into horizontal and vertical wires

In [ ]:
def split_diagonal_segments(wire_list):
    new_wire_list = []
    for segment in wire_list:
        # Check if the segment is diagonal
        if segment['x'] != segment['end_x'] and segment['y'] != segment['end_y']:
            # Split the diagonal segment into horizontal and vertical segments
            horizontal_segment = {'x': segment['x'], 'y': segment['y'], 'end_x': segment['end_x'], 'end_y': segment['y']}
            vertical_segment = {'x': segment['end_x'], 'y': segment['y'], 'end_x': segment['end_x'], 'end_y': segment['end_y']}
            new_wire_list.append(horizontal_segment)
            new_wire_list.append(vertical_segment)
        else:
            new_wire_list.append(segment)  # Keep non-diagonal segments unchanged
    return new_wire_list


new_wire_list = split_diagonal_segments(wire_list)
print(new_wire_list)

# Add wires to schematic!

In [ ]:
from scripts import kicad_utils
kicad_schematic_path = "testProject/testProject.kicad_sch"
kicad_utils.modify_kicad_sch_file(wires= new_wire_list, file_path=kicad_schematic_path)

In [11]:
import yaml
# Path to your config.yaml file
config_file_path = 'configuration.yaml'

# Read the YAML file
with open(config_file_path, 'r') as file:
    config = yaml.safe_load(file)

# Extract the OPENAI_API_KEY value
symbol_library_path = config.get('symbol_library_path', None)
print(symbol_library_path)

/usr/share/kicad/symbols/


### R&D
#### Try to get symbols in a json and then search for unindentified symbols in the llm output

In [43]:
import os
import re
import json

def extract_top_level_symbol_names(file_path):
    symbol_names = []
    with open(file_path, "r") as f:
        content = f.read()
        symbol_matches = re.findall(r'\(symbol\s+"([^"]+)"', content)
        previous_symbol = ""
        for symbol_match in symbol_matches:
            escaped_symbol = re.escape(previous_symbol) if previous_symbol else ""
            if bool(re.search(fr'{escaped_symbol}_\d+', symbol_match)):
                continue
            else:
                previous_symbol = symbol_match
                symbol_names.append(symbol_match)
    return symbol_names

def extract_and_write_symbol_data(directory_path, output_file):
    with open(output_file, "w") as json_file:
        json_file.write('{"symbols": [')
        first_symbol = True
        for filename in os.listdir(directory_path):
            if filename.endswith(".kicad_sym"):
                # Extract the library name from the file name
                libname= filename.split(".")[0]
                file_path = os.path.join(directory_path, filename)
                symbols = extract_top_level_symbol_names(file_path)
                if symbols:
                    if not first_symbol:
                        json_file.write(',')
                    json.dump({"lib": filename, "symbols": symbols}, json_file, indent=2)
                    first_symbol = False
        json_file.write(']}')

# Specify the directory containing the KiCad symbol files
directory_path = "/usr/share/kicad/symbols"

# Specify the output file
output_file = "symbol_data.json"

# Extract and write symbol data
extract_and_write_symbol_data(directory_path, output_file)


### Find file in json

In [54]:
# Naive Search

import json

def search_symbol(json_data, symbol_name):
    for symbol_file in json_data["symbols"]:
        if symbol_name in symbol_file["symbols"]:
            return symbol_file["lib"]
    return None

# Load JSON data from file (assuming data is stored in "data.json")
with open("symbol_data.json", "r") as f:
    json_data = json.load(f)

# Search for symbol "symbolC"
symbol_to_find = "STM"
symbol_file_found = search_symbol(json_data, symbol_to_find)

if symbol_file_found:
    print(f"Symbol '{symbol_to_find}' found in file: {symbol_file_found}")
else:
    print(f"Symbol '{symbol_to_find}' not found.")

Symbol 'STM' not found.


In [52]:
# Levenshtein Search

import json
import Levenshtein

def load_symbol_data(file_path):
    with open(file_path, "r") as f:
        symbol_data = json.load(f)
    return symbol_data

def find_closest_matches(term, symbol_data, top_n=3):
    matches = []
    for lib_data in symbol_data["symbols"]:
        lib_name = lib_data["lib"]
        symbols = lib_data["symbols"]
        for symbol in symbols:
            distance = Levenshtein.distance(term.lower(), symbol.lower())
            matches.append((lib_name, symbol, distance))
    
    # Sort the matches based on distance
    sorted_matches = sorted(matches, key=lambda x: x[2])
    
    # Extract top N matches
    top_matches = sorted_matches[:top_n]
    
    return top_matches

# Load symbol data from symbol.json
symbol_data_path = "symbol_data.json"
symbol_data = load_symbol_data(symbol_data_path)

# Example usage:
term = "Batt"
top_matches = find_closest_matches(term, symbol_data)
for i, match in enumerate(top_matches, start=1):
    print(f"Match {i}: Library: {match[0]}, Symbol: {match[1]}, Distance: {match[2]}")


Match 1: Library: power.kicad_sym, Symbol: +BATT, Distance: 1
Match 2: Library: power.kicad_sym, Symbol: -BATT, Distance: 1
Match 3: Library: Diode.kicad_sym, Symbol: BAT41, Distance: 2
